<a href="https://colab.research.google.com/github/bishair/Pirna/blob/main/ModelPirna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and upload excel files for groundwater level and river water level separately

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [9]:
uploaded_file = files.upload()

Saving riverL.xlsx to riverL.xlsx


# Main Functions
**merge_and_resample:**

Merge and preprocess data from two Excel files containing groundwater and river water level data along with their respective dates, function  reads the two files, merges the data on the date column, resamples it to an hourly frequency, and then applies forward fill to handle missing hours.

**preprocess_data_for_lstm:**

Handle missing values for groundwater level and river water level by using the previous value. Normalize the data in the range 0 to 1. Return numpy arrays for X and y and create sequences using windows of given length.

**Train and save model:**
Specify optimizer, loss function, batch size, epochs and number of neurons and layers for model training

**detect_anomalies**
The function uses a moving window approach, where the size of the window is defined by the window_size parameter. Within this moving window, it calculates the mean and standard deviation of 'G10' values. Anomalies are detected based on a z-score threshold, which is set to 2.0 by default. The z-score represents how many standard deviations a data point is from the moving average. Points where the absolute value of the z-score exceeds the specified threshold are flagged as anomalies. The function  returns the original DataFrame with an additional 'Anomaly' column, marking true where anomalies are detected in 'G10'.

**preprocess_data_for_retraining**
The preprocess_data_for_retraining function is used for preprocessing data for LSTM models, focusing on predicting the 'G10' variable using its past values along with 'River' variable. It accepts a pandas DataFrame containing these two columns and a parameter n_steps, which specifies the number of past time steps to consider for predicting future values. The function involves selecting relevant columns, handling missing values via forward filling, normalizing the data using MinMaxScaler, and creating sequential datasets suitable for LSTM. These sequences are constructed using the specified n_steps, forming feature and label sets (X and y). Finally, the data is split into training and testing sets,  70-30 split ratio, and the function returns these sets as numpy arrays for input to the lstm model

In [10]:
#Use consistent time data, in case of missing observations use forward fill to have consistent time data
#date_format= '%Y-%m-%d %H:%M:%S'

def merge_and_resample(file_groundwater, file_river, date_format_groundwater, date_format_river):
    # Read the groundwater level data
    gw_data = pd.read_excel(file_groundwater)
    gw_data['Date'] = pd.to_datetime(gw_data['Date'], format=date_format_groundwater)

    # Read the river water level data
    river_data = pd.read_excel(file_river)
    river_data['Date'] = pd.to_datetime(river_data['Date'], format=date_format_river)

    # Merge the two datasets on the 'Date' column
    merged_data = pd.merge_asof(gw_data.sort_values('Date'), river_data.sort_values('Date'), on='Date', direction='nearest')

    # Set 'Date' as the index
    merged_data.set_index('Date', inplace=True)

    # Create a new date range that starts from the first timestamp in merged_data
    start_date = merged_data.index.min()
    end_date = merged_data.index.max()
    new_date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    # Reindex and resample merged_data with the new date range
    resampled_data = merged_data.reindex(new_date_range).ffill().bfill()

    return resampled_data



In [11]:

def preprocess_data_for_lstm(data, n_steps):
    """
    Preprocess the data for LSTM model to predict 'G10' using past values of 'G10' and 'River'.
    - data: pandas DataFrame containing 'G10' and 'River' columns.
    - n_steps: number of time steps to use for predicting the next time step.

    Returns:
    - X, y: numpy arrays of features and labels suitable for LSTM model.
    """
    # Handling missing values using forward fill
    data = data.fillna(method='ffill')

    # Normalizing the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    # Creating sequences
    X, y = [], []
    for i in range(n_steps, len(data_scaled)):
        X.append(data_scaled[i-n_steps:i, :])
        y.append(data_scaled[i, 0])

    X, y = np.array(X), np.array(y)
    # Split the data into training and testing sets
    train_size = int(len(X) * 0.7)
    test_size = len(X) - train_size
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test



In [12]:
def train_model(x_train, y_train, input_shape, epochs=100, batch_size=32):
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        LSTM(units=50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

In [13]:
def save_model(model, filename='my_lstm_model.h5'):
    model.save(filename)
    return model

In [14]:
def evaluate_model(model, x_test, y_test):
    # Evaluate the model's performance on the test data
    # It returns a list: [loss, mse]
    loss = model.evaluate(x_test, y_test)
    return loss


In [22]:
def preprocess_data_for_retraining(data, n_steps):
    """
    Preprocess the data for LSTM model to predict 'G10' using past values of 'G10' and 'River'.
    - data: pandas DataFrame containing 'G10' and 'River' columns.
    - n_steps: number of time steps to use for predicting the next time step.

    Returns:
    - X, y: numpy arrays of features and labels suitable for LSTM model.
    """
    # Select only relevant columns
    relevant_data = data[['G10', 'River']].copy()

    # Handling missing values using forward fill
    relevant_data.fillna(method='ffill', inplace=True)

    # Normalizing the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(relevant_data)


    # Creating sequences
    X, y = [], []
    for i in range(n_steps, len(data_scaled)):
        X.append(data_scaled[i-n_steps:i, :])
        y.append(data_scaled[i, 0])

    X, y = np.array(X), np.array(y)
    # Split the data into training and testing sets
    train_size = int(len(X) * 0.7)
    test_size = len(X) - train_size
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test

In [23]:
def detect_anomalies(data, window_size, threshold=2.0):
    """
    Detects anomalies in the 'G10' column of a time series data using a moving window method.

    - data: pandas DataFrame with a datetime index and at least a 'G10' column for groundwater levels.
    - window_size: The size of the moving window.
    - threshold: The z-score threshold to identify an anomaly.

    Returns a DataFrame with an 'Anomaly' column indicating True where anomalies were detected in 'G10'.
    """
    # Calculate the moving average and standard deviation for 'G10'
    rolling_mean = data['G10'].rolling(window=window_size).mean()
    rolling_std = data['G10'].rolling(window=window_size).std()

    # Calculate the z-score for each data point in 'G10'
    data['Z-Score'] = (data['G10'] - rolling_mean) / rolling_std

    # Identify data points where the absolute z-score exceeds the threshold
    data['Anomaly'] = abs(data['Z-Score']) > threshold

    return data

# Forward fill the missing hours by calling merge_and_resample function

In [15]:
merged_df = merge_and_resample('groundwater.xlsx','riverL.xlsx', '%d/%m/%Y %H:%M' ,'%d/%m/%Y %H:%M')

# Preprocess and define window size for LSTM model by using function(preprocess_data_for_lstm)

In [16]:
n_steps = 24
X_train, y_train, X_test, y_test = preprocess_data_for_lstm(merged_df, n_steps)

# Train and save model

In [17]:
"""
input_shape  includes two dimensions: (time_steps, features)
Time Steps: Number of time intervals or sequences each input sample consists of.
Features: Number of variables observed at each time step.
"""

input_shape=(X_train.shape[1], X_train.shape[2])
epochs=40
batch_size=64
trained_model = train_model(X_train, y_train, input_shape, epochs, batch_size)

Epoch 1/40
192/192 [==============================] - 12s 40ms/step - loss: 2.4066e-04
Epoch 2/40
192/192 [==============================] - 7s 35ms/step - loss: 7.8798e-06
Epoch 3/40
192/192 [==============================] - 8s 41ms/step - loss: 6.8063e-06
Epoch 4/40
192/192 [==============================] - 7s 35ms/step - loss: 1.4176e-05
Epoch 5/40
192/192 [==============================] - 7s 38ms/step - loss: 8.8629e-06
Epoch 6/40
192/192 [==============================] - 6s 33ms/step - loss: 8.2022e-06
Epoch 7/40
192/192 [==============================] - 8s 39ms/step - loss: 7.2290e-06
Epoch 8/40
192/192 [==============================] - 6s 30ms/step - loss: 1.0168e-05
Epoch 9/40
192/192 [==============================] - 8s 39ms/step - loss: 4.8136e-06
Epoch 10/40
192/192 [==============================] - 6s 31ms/step - loss: 1.3886e-05
Epoch 11/40
192/192 [==============================] - 8s 40ms/step - loss: 9.7894e-06
Epoch 12/40
192/192 [==============================

In [18]:
saved_model = save_model(trained_model)
print(saved_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Evaluate Model by calculating MSE

In [19]:
evaluate_model(saved_model, X_test, y_test)


165/165 [==============================] - 3s 10ms/step - loss: 1.1820e-07


1.1820337419976568e-07

# ------------------------**Anomaly detection in new data**------------------------------

In [25]:
uploaded_file = files.upload()

Saving river_a.xlsx to river_a.xlsx


In [27]:
# Merge two new excel files and resampled to have consistent hourly dates
merged_df_new = merge_and_resample('gw_a.xlsx','river_a.xlsx', '%d/%m/%Y %H:%M' ,'%d/%m/%Y %H:%M')

In [31]:
#Call detect anomalies function and check for any anomalies in new data. This function will result in a dataframe containing z-score values and boolean for anomaly values
anomaly_df = detect_anomalies(merged_df_new, 3, threshold=1)

In [35]:
# Check for anomalies
if anomaly_df['Anomaly'].any():
    # Merge old and new data
    combined_data = pd.concat([merged_df, merged_df_new])
    # Preprocess the combined data for retraining
    X_train, y_train, X_test, y_test = preprocess_data_for_retraining(combined_data, n_steps)

    # Define the input shape for the LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])

    # Retrain the model
    trained_model = train_model(X_train, y_train, input_shape, epochs=40, batch_size=64)

    # Evaluate and save the retrained model
    saved_model = save_model(trained_model)
    print("Model retrained due to anomaly detection.")
    evaluate_model(saved_model, X_test, y_test)
else:
    print("No anomalies detected, no retraining required.")


Epoch 1/40
364/364 [==============================] - 18s 35ms/step - loss: 0.0031
Epoch 2/40
364/364 [==============================] - 14s 38ms/step - loss: 3.5876e-05
Epoch 3/40
364/364 [==============================] - 13s 36ms/step - loss: 2.4244e-05
Epoch 4/40
364/364 [==============================] - 13s 37ms/step - loss: 2.3736e-05
Epoch 5/40
364/364 [==============================] - 13s 36ms/step - loss: 2.2133e-05
Epoch 6/40
364/364 [==============================] - 13s 36ms/step - loss: 2.4120e-05
Epoch 7/40
364/364 [==============================] - 14s 38ms/step - loss: 2.4847e-05
Epoch 8/40
364/364 [==============================] - 13s 36ms/step - loss: 2.2990e-05
Epoch 9/40
364/364 [==============================] - 13s 35ms/step - loss: 2.2764e-05
Epoch 10/40
364/364 [==============================] - 14s 37ms/step - loss: 2.2445e-05
Epoch 11/40
364/364 [==============================] - 13s 37ms/step - loss: 2.3590e-05
Epoch 12/40
364/364 [========================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model retrained due to anomaly detection.
312/312 [==============================] - 4s 10ms/step - loss: 2.4797e-07
